# Greenland peripherial glacier terminus image processing

### Jukes Liu

## 1) Import packages, set base path, set glaciers of interest by BoxID

In [5]:
import subprocess
import numpy as np
import os
import pandas as pd
import rasterio

#SET basepath to your own folder
basepath='/Users/julialiu/Documents/M_Thesis/Data/Sample_glaciers/'

#ENTER list of glaciers of interest by BoxID
#make this into a widget where you can enter them in?
BOXIDS = ['001', '002', '004', '033', '120', '174', '235', '259', '277', '531'];

ModuleNotFoundError: No module named 'numpy'

## 2) Create buffer zone around terminus boxes and rasterize/subset terminus boxes

The following code pulls the buffer distances around the terminus boxes from an existing .csv file with the exported attributes tables for the peripheral glacier terminus boxes. These buffer distances will be used to create a buffer zone to subset the Landsat scenes.

In [112]:
#PULL buffer distances around terminus boxes from a csv file of attributes
df=pd.read_csv(basepath+'Boxes_attributes.csv', sep=','); #reads in csv file 
df_buffdist = df["Buff_dist"].copy(); #creates a dataframe of the buffer distances

df_boxid = df["BoxID"].copy(); #creates a data frame of the BoxIDs
df_b = pd.concat([df_buffdist, df_boxid], axis=1); #concatenates the two columns

#CREATE dictionary of buffer distances with BoxID as the key
bd = df_b.set_index('BoxID').T.to_dict('list'); #turns the concatenated df into a dictionary

#PRINT buffer distances for the glaciers of interest
for BoxID in BOXIDS: 
    bd_key = int(BoxID)
    buff_dist = str(int(bd[bd_key][0]))
    print("Box"+BoxID+":", buff_dist, "meters")


Box001: 1766 meters
Box002: 6741 meters
Box004: 7112 meters
Box033: 5424 meters
Box120: 1720 meters
Box174: 3729 meters
Box235: 2620 meters
Box259: 6974 meters
Box277: 2273 meters
Box531: 2657 meters


The next section creates a buffer zone using GDAL command **ogr2ogr** with the following syntax:

    ogr2ogr Buffer###.shp path_to_terminusbox###.shp  -dialect sqlite -sql "SELECT ST_Buffer(geometry, buffer_distance) AS geometry,*FROM 'Box###'" -f "ESRI Shapefile"

In [118]:
#export GDAL path command:
export_GDALpath = 'export PATH=/Library/Frameworks/GDAL.framework/Programs:$PATH ; '

for BoxID in BOXIDS:
    #SET path to the terminus box shapefiles
    terminusbox_path = basepath+"Box"+BoxID+"/Box"+BoxID+".shp"
    outputbuffer_path = basepath+"Box"+BoxID+"/Buffer"+BoxID+".shp"
    
    #PULL the buffer distances as strings from the bd dictionary using the BoxID keys
    bd_key = int(BoxID)
    buff_dist = str(int(bd[bd_key][0]))
    
    #SET buffer command and print to check it
    buffer_cmd = 'ogr2ogr '+outputbuffer_path+" "+terminusbox_path+' -dialect sqlite -sql "SELECT ST_Buffer(geometry, '+buff_dist+") AS geometry,*FROM 'Box"+BoxID+"'"+'" -f "ESRI Shapefile"'
    #print(export_GDALpath, buffer_cmd)
    
    subprocess.call(export_GDALpath+buffer_cmd, shell=True)
    
    print("Box"+BoxID)


Box001
Box002
Box004
Box033
Box120
Box174
Box235
Box259
Box277
Box531


The terminus box shapefiles are then rasterized (to be used as a mask during the WTMM filering) using the GDAL **gdal_rasterize** command and subset to the buffer zone using the GDAL **gdalwarp** command using the following syntax:

1) Rasterize

    gdal_rasterize -burn 1.0 -tr x_resolution y_resolution -a_nodata 0.0 path_to_terminusbox.shp path_to_terminusbox_raster.TIF

The x_resolution and y_resolution are set to be 15.0 (meters) to match the Landsat B8 resolution.
    
2) Subset

    gdalwarp -cutline path_to_Buffer###.shp -crop_to_cutline path_to_terminusbox_raster.TIF path_to_subset_raster_cut.TIF

In [141]:
#export GDAL path command:
export_GDALpath = 'export PATH=/Library/Frameworks/GDAL.framework/Programs:$PATH ; '

for BoxID in BOXIDS:
    #SET path to the terminus box shapefiles
    terminusbox_path = basepath+"Box"+BoxID+"/Box"+BoxID+".shp"
    buffer_path = basepath+"Box"+BoxID+"/Buffer"+BoxID+".shp"
    
    #output raster path:
    terminusraster_path = basepath+"Box"+BoxID+"/Box"+BoxID+".TIF"
    cutraster_path = basepath+"Box"+BoxID+"/Box"+BoxID+"_cut.TIF"
    
    #SET commands and print to check
    rasterize_cmd = 'gdal_rasterize -burn 1.0 -tr 15.0 15.0 -a_nodata 0.0 '+terminusbox_path+' '+terminusraster_path
    subsetbuffer_cmd = 'gdalwarp -cutline '+buffer_path+' -crop_to_cutline '+terminusraster_path+" "+cutraster_path
    #print(export_GDALpath+rasterize_cmd)
    #print(export_GDALpath+subsetbuffer_cmd)
    
    #RASTERIZE & SUBSET
    subprocess.call(export_GDALpath+rasterize_cmd, shell=True)
    subprocess.call(export_GDALpath+subsetbuffer_cmd, shell=True)
    
    print("Box"+BoxID)
    

Box001
Box002
Box004
Box033
Box120
Box174
Box235
Box259
Box277
Box531


## 3) Subset downloaded Landsat scenes

In [115]:
#READ in the Landsat path and row information for the glaciers
pathrows_df = pd.read_csv('/Users/julialiu/Documents/M_Thesis/Data/LS_pathrows.csv', sep=',', usecols =[0,1,2], dtype=str, nrows =10)
pathrows_df = pathrows_df.set_index('BoxID')
pathrows_df.head()

,Path,Row
BoxID,,
001,034,005
002,031,005
004,031,005
033,008,014
120,232,017


The following section of code accesses the Landsat 8 scenes downloaded and stored in Path###_Row### folders to 1) reproject them into Greenland Polar Stereographic Coordinates and 2) subset them using the buffer zone shapefiles created from the previous step. The two GDAL commands (**gdalwarp**) use the following syntax:

1) Reproject

    gdalwarp -t_srs ‘+proj=stere +lat_ts=70 +lat_0=90 +lon_0=-45 +y=0 +x=0 +k=1 +datum=WGS84 +units=m’ path_to_input_image.TIF path_to_renamed_output.TIF

2) Subset

    gdalwarp -cutline _path_to_Buffer###.shp -crop_to_cutline path_to_input_image.TIF path_to_renamed_output.TIF

In [138]:
#set path to LS8 scenes
path_toscenedirectory = '/Users/julialiu/Documents/M_Thesis/Data/LS8aws/'

#export GDAL path command:
export_GDALpath = 'export PATH=/Library/Frameworks/GDAL.framework/Programs:$PATH ; '

for BoxID in BOXIDS:
    
    #SELECT folder with LS8 scenes for each glacier by Path and Row folder name
    path = pathrows_df.loc[BoxID, 'Path']
    row = pathrows_df.loc[BoxID, 'Row']
    foldername = "Path"+path+"_Row"+row
    path_toscenes = path_toscenedirectory+foldername+"/"
    #print(path_toscenes)
    
    #set path to buffer zone
    buffer_path = basepath+"Box"+BoxID+"/Buffer"+BoxID+".shp"
    
    #STEPS for each image:
    for image in os.listdir(path_toscenes):
        #REPROJECT into Greenland Polar Stereographic
        reproject_cmd = "gdalwarp -t_srs +proj=stere +lat_ts=70 +lat_0=90 +lon_0=-45 +y=0 +x=0 +k=1 +datum=WGS84 +units=m "+path_toscenes+image+"/"+image+"_B8.TIF "+path_toscenes+image+"/"+image+"_B8_PS.TIF"
        #print(export_GDALpath+reproject_cmd)
        #subprocess.call(export_GDALpath+reproject_cmd, shell=True)

        #SUBSET LS8 scenes to buffer zones around the terminus box
        #save subsetted scenes as PGM files
        subsetbuffer_cmd = 'gdalwarp -cutline '+buffer_path+' -crop_to_cutline '+path_toscenes+image+"/"+image+"_B8_PS.TIF "+path_toscenes+image+"/"+image+"_B8_PS_Buffer"+BoxID+".pgm"
        #print(export_GDALpath+subsetbuffer_cmd)
        #subprocess.call(export_GDALpath+subsetbuffer_cmd)
        
    print("Box"+BoxID)
        
        

Box001
Box002
Box004
Box033
Box120
Box174
Box235
Box259
Box277
Box531


## 4) Calculate average flow direction (weighted by magnitude) for each glacier

The following code processes 2016-2017 ice velocity data from the ESA Cryoportal to determine each glacier of interest's weighted average flow direction. The ice velocity direction (calculated from yx velocity) and the velocity magnitude at each glacier's terminus  is subset using the terminus box shapefile using a GDAL command (**gdalwarp**) with the following syntax:

    gdalwarp -cutline path_to_terminusbox.shp -crop_to_cutline path_to_input_velocity.TIF path_to_output_velocity_at_term###.TIF

In [135]:
#export GDAL path command:
export_GDALpath = 'export PATH=/Library/Frameworks/GDAL.framework/Programs:$PATH ; '

for BoxID in BOXIDS:
    #SET paths to the terminus box shapefiles and velocity data
    terminusbox_path = basepath+"Box"+BoxID+"/Box"+BoxID+".shp"
    v_dir_path = basepath+'dir_degree_yx_velocity.tif'
    v_mag_path = basepath+'magnitude_velocity.tif'  
    
    #SET output paths
    v_dir_output = basepath+"Box"+BoxID+"/dir_degree_yx_velocity_at_term"+BoxID+".tif"
    v_mag_output = basepath+"Box"+BoxID+"/magnitude_velocity_at_term"+BoxID+".tif"
    
    #SET velocity subset commands and print to check it
    v_subset_dir_cmd = 'gdalwarp -cutline '+terminusbox_path+' -crop_to_cutline '+v_dir_path+" "+v_dir_output
    v_subset_mag_cmd = 'gdalwarp -cutline '+terminusbox_path+' -crop_to_cutline '+v_mag_path+" "+v_mag_output
    #print(export_GDALpath+v_subset_dir_cmd)
    #print(export_GDALpath+v_subset_mag_cmd)
    
    #SUBSET velocity rasters
    subprocess.call(export_GDALpath+v_subset_dir_cmd, shell=True)
    subprocess.call(export_GDALpath+v_subset_mag_cmd, shell=True)
    
    print("Box"+BoxID)

Box001
Box002
Box004
Box033
Box120
Box174
Box235
Box259
Box277
Box531


Next, these subset velocity rasters are opened using the **rasterio** package and read into arrays. They are filtered for anomalous values and the velocity magnitudes are converted into weights. Then the **numpy.average()** function is used to calculated the weighted average flow directions where the flow directions of the pixels where the highest velocities are found are weighted more. 

The resulting average flow direction will be representative of the glacier's main flow. These directions will be used to rotate the images of the glaciers so that their flow is due right.

In [6]:
#CREATE list of glacier average flow directions:
rot_angles = {}
max_magnitudes = {}

for BoxID in BOXIDS :    
        #READ velocity direction and magnitude data at terminus for each glacier into an array
        direction = rasterio.open(basepath+'Box'+BoxID+'/dir_degree_yx_velocity_at_term'+BoxID+'.tif', "r")
        dir_array = direction.read()
        #print(dir_array.shape)
        magnitude = rasterio.open(basepath+'Box'+BoxID+'/magnitude_velocity_at_term'+BoxID+'.tif', "r")
        mag_array = magnitude.read()
        #print(mag_array.shape)
        
        
        #RESHAPE direction array and remove anomalous values
        dir2 = dir_array.reshape(dir_array.shape[1] * dir_array.shape[2])
        #direction must be between 180 and -180 degrees
        mask_dir2 = (dir2 < 180) & (dir2 > -180)
        masked_dir = dir2[mask_dir2]
        #print(masked_dir.min(), masked_dir.max())

        #RESHAPE magnitude array and remove anomalous values
        mag2 = mag_array.reshape(mag_array.shape[1] * mag_array.shape[2])
        #magnitude must be between 0 and 10 m/d
        mask_mag2 = (mag2 < 25) & (mag2 > 0)
        masked_mag = mag2[mask_mag2]
        #print(masked_mag.min(), masked_mag.max())
        
    
        #CALCULATE weights (0 - 1) from magnitudes
        mag_range = masked_mag.max() - masked_mag.min()
        stretch = 1/mag_range
        weights = stretch*(masked_mag - masked_mag.min())
        #print(weights.min(), weights.max()) #should be between 0 and 1
        #print(weights.shape, masked_dir.shape)
        
        
        #CALCULATE the weighted average rotation angle
        avg_dir = np.average(masked_dir, weights=weights)
        #print(avg_dir)
        
        #APPEND the rotation angles to the dictionary
        rot_angles.update( {BoxID: avg_dir})
        
        #APPEND the maximum flow magnitude
        max_magnitudes.update( {BoxID: masked_mag.max()} )


print(rot_angles)
print(max_magnitudes)

#EXPORT TO CSV for rotations???

{'001': 56.284283, '002': 155.98723, '004': -3.4834337, '033': 142.11816, '120': -77.3864, '174': 12.677643, '235': -145.85077, '259': 98.999275, '277': -65.120186, '531': 78.83521}
{'001': 0.04377438, '002': 3.5832264, '004': 0.62308246, '033': 0.77165776, '120': 0.27788857, '174': 0.91450316, '235': 0.15709679, '259': 3.0749009, '277': 0.2860196, '531': 0.040784776}


## 5) Call FIJI scripts to perform rotations, etc.

In [31]:
#NEXT, USE FIJI/IMAGEJ TO PERFORM THE ROTATIONS, use iPython line or cell magics

## 6) Call Tcl scripts to perform 2D WTMM and filter for terminus line

## 7) Call post-processing scripts